In [3]:
!pip install pandas transformers torch

In [4]:
import pandas as pd

# Load the dataset
df = pd.read_csv("C:/Users/nguye/OneDrive/Tài liệu/GitHub/21KHDL-TikTok-Analytics/data/interim/video_info.csv")

# Drop rows with empty text
df = df.dropna(subset=['text'])

KeyError: ['text']

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base-vietnews-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base-vietnews-summarization")

def summarize(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(**inputs, max_length=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Add a summary column to the DataFrame
df["summary"] = df["text"].apply(summarize)

In [ ]:
df.to_csv("audio_text_processed.csv", index=False)